This model remove "Entity:" and "Sentence:" in the input 10 epoch

In [1]:
import pandas as pd
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import gc

comet_ml is installed but `COMET_API_KEY` is not set.


In [2]:
#Load dataset
df = pd.read_csv('/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Database Part/manual_label - consolidated.csv')

In [3]:
# Check dataset
df.head()

,entity,sentence,paragraph,sentence_original,label,class_ID
0,retailer,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
1,Proposition 1,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,belongs_to_article,1
2,Supply-Chain,which gives the supplier 's profit as and the ...,which gives the supplier's profit as and the r...,which gives the supplier's profit as and the r...,background_information,3
3,Petri net,which can be equivalently represented with a P...,which can be equivalently represented with a P...,which can be equivalently represented with a P...,background_information,3
4,business processes,which are sent by business processes accounts ...,A simple example of a synthesis process for IA...,which are sent by business processes accounts ...,background_information,3


In [4]:
# Extract needed columns
df = df[['entity', 'sentence_original', 'class_ID']]
df.head()

,entity,sentence_original,class_ID
0,retailer,which gives the supplier's profit as and the r...,3
1,Proposition 1,which gives the supplier's profit as and the r...,1
2,Supply-Chain,which gives the supplier's profit as and the r...,3
3,Petri net,which can be equivalently represented with a P...,3
4,business processes,which are sent by business processes accounts ...,3


In [5]:
# Combine entity and sentence into one text column
df['text'] = df.apply(lambda row: f"{row['entity']}, {row['sentence_original']}", axis=1)

# Rename 'class_ID' to 'label' to match SetFit expectations
df = df.rename(columns={'class_ID': 'label'})

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert the DataFrame to a Dataset object
train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
test_dataset = Dataset.from_pandas(test_df[['text', 'label']])

In [6]:
train_df

,entity,sentence_original,label,text
38,cybercrime,whereas the magnitude of the coefficient of fi...,1,"cybercrime, whereas the magnitude of the coeff..."
143,framework,word2vec method is a self-supervised framework...,3,"framework, word2vec method is a self-supervise..."
84,organizational level,"yuqing Ren, Sara kiesler, and Susan R. Fussell...",2,"organizational level, yuqing Ren, Sara kiesler..."
55,online survey,"zoomeRaNG GeNeRated a RaNdom set of 1,200 sale...",1,"online survey, zoomeRaNG GeNeRated a RaNdom se..."
217,evaluation of Internet,whose primary goal is to produce theory driven...,3,"evaluation of Internet, whose primary goal is ..."
...,...,...,...,...
106,CFI was 0.924,"x 2 /df was 3.67, which was below the acceptab...",1,"CFI was 0.924, x 2 /df was 3.67, which was bel..."
14,individual-level,whether the predictors display curvilinear eff...,2,"individual-level, whether the predictors displ..."
92,information technology,"yet, as researchers, we do not often delve int...",3,"information technology, yet, as researchers, w..."
179,British,with other international airlines such as Qant...,3,"British, with other international airlines suc..."


In [7]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 202
})

In [8]:
test_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 51
})

In [9]:
from setfit import SetFitModel, Trainer, sample_dataset
# Initialize the SetFit model
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [10]:
# No enough memory to run this, thus switching to cpu
import torch

device = torch.device("cpu")
model.to(device)

SetFitModel(model_body=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_head=LogisticRegression(), multi_target_strategy=None, normalize_embeddings=False, labels=None, model_card_data=SetFitModelCardData(language=None, license=None, tags=['setfit', 'sentence-transformers', 'text-classification', 'generated_from_setfit_trainer'], model_name='SetFit with sentence-transformers/paraphrase-mpnet-base-v2', model_id=None, dataset_name=None, dataset_id=None, dataset_revision=None, task_name=None, st_id='sentence-transformers/paraphrase-mpnet-base-v2', hyperparameters={}, eval_results_dict={}, eval_lines_list=[], metric_l

In [11]:
import torch
import gc
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitTrainer
import psutil



# Create a Trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss_class=CosineSimilarityLoss,
    num_epochs=10
)

# Train the model
trainer.train()



/var/folders/ld/mczz0xmx243grk32lg665c_c0000gn/T/ipykernel_47631/2195367269.py:10: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/202 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8080
  Batch size = 16
  Num epochs = 10
  Total optimization steps = 5050


  0%|          | 0/5050 [00:00<?, ?it/s]

  0%|          | 0/5050 [00:00<?, ?it/s]

{'embedding_loss': 0.3722, 'learning_rate': 3.960396039603961e-08, 'epoch': 0.0}
{'embedding_loss': 0.2741, 'learning_rate': 1.9801980198019803e-06, 'epoch': 0.1}
{'embedding_loss': 0.2237, 'learning_rate': 3.960396039603961e-06, 'epoch': 0.2}
{'embedding_loss': 0.2428, 'learning_rate': 5.940594059405941e-06, 'epoch': 0.3}
{'embedding_loss': 0.2085, 'learning_rate': 7.920792079207921e-06, 'epoch': 0.4}
{'embedding_loss': 0.3421, 'learning_rate': 9.900990099009901e-06, 'epoch': 0.5}
{'embedding_loss': 0.0882, 'learning_rate': 1.1881188118811881e-05, 'epoch': 0.59}
{'embedding_loss': 0.0499, 'learning_rate': 1.3861386138613861e-05, 'epoch': 0.69}
{'embedding_loss': 0.009, 'learning_rate': 1.5841584158415843e-05, 'epoch': 0.79}
{'embedding_loss': 0.0017, 'learning_rate': 1.7821782178217823e-05, 'epoch': 0.89}
{'embedding_loss': 0.0008, 'learning_rate': 1.9801980198019803e-05, 'epoch': 0.99}
{'embedding_loss': 0.0006, 'learning_rate': 1.9801980198019803e-05, 'epoch': 1.09}
{'embedding_loss

In [12]:
trainer.model.save_pretrained('/Users/jinlinchen/Documents/Study/HWR Berlin/Semester 2/Analytics Lab/Analytics Project/Database part/model/model6/model')

In [15]:
# Evaluate the model on the test set
results = trainer.evaluate()
print(f"Evaluation results: {results}")

***** Running evaluation *****


Evaluation results: {'accuracy': 0.7058823529411765}


In [16]:
from sklearn.metrics import classification_report
from datasets import load_metric

# Load the metric
metric = load_metric("f1")

# Get predictions from the model
preds = model(test_dataset["text"])
labels = test_dataset["label"]

# Calculate F1 score
f1_score = metric.compute(predictions=preds, references=labels, average="weighted")
print(f"F1 score: {f1_score['f1']}")

# Define class names manually
class_names = ["1", "2", "3"]  # Replace with your actual class names

# Generate classification report
report = classification_report(labels, preds, target_names=class_names)
print(report)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for f1 contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/f1/f1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


F1 score: 0.7010311648599248
              precision    recall  f1-score   support

           1       0.67      0.89      0.76        18
           2       0.62      0.56      0.59         9
           3       0.79      0.62      0.70        24

    accuracy                           0.71        51
   macro avg       0.69      0.69      0.68        51
weighted avg       0.72      0.71      0.70        51

